In [314]:
import numpy as np
from numba import jit
import time
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import math
from scipy.stats import norm

## Part II: Estimation of Sensitivities in MC

In [310]:
def Delta_Analytical(S, K, r, sigma, tau):
    d1 = (math.log(S/K) + (r + 0.5 * sigma**2) * tau) / (sigma * math.sqrt(tau))
    return norm.cdf(d1)

In [380]:
@jit(nopython=True, parallel=True)
def European_Put_Delta(T, K, r, S, sigma, trials, e, seed_target=None):
    '''
    This function calculates the value of an European Put option
    Arguments: maturity, strike price, interest rate, stock price, volaility, number of trials
    Returns: Array of size trial with values of european puts
    '''
    Se = S + e
    S_adjust = S * np.exp(r - (0.5 * sigma**2)*T)
    Se_adjust = Se * np.exp(r - (0.5 * sigma**2)*T)
    
    delta_array = np.zeros(trials)
    
    for i in range(trials):
        if seed_target != None:
            np.random.seed(seed_target)
            rand = np.random.normal()
            rand_1 = rand
        else:
            rand=np.random.normal()
            rand_1=np.random.normal()
            
            
        S_cur = S_adjust * np.exp(sigma*np.sqrt(T)*rand
        Se_cur = Se_adjust * np.exp(sigma*np.sqrt(T)*rand_1
        
        if K-S_cur > 0:
            payoff = (K-S_cur)*np.exp(-r*T)
        else:
            payoff = 0
            
        if K-Se_cur > 0:
            payoff_e = (K-Se_cur)*np.exp(-r*T)
        else:
            payoff_e = 0
            
        delta_array[i] = (payoff_e - payoff)/e

    return delta_array

SyntaxError: invalid syntax (<ipython-input-380-a2abfadb33cf>, line 25)

In [374]:
T = 1
K = 99
r=0.06
S=100
sigma=0.2
trials=10**7
e=0.5
seed_target=100
delta_array = European_Put_Delta(T,K,r,S,sigma,trials,e,seed_target)

In [375]:
Delta_Analytical(S, K, r, sigma, 0.5)

0.6382965784784819